In [1]:
from evo.notebooks import ServiceManagerWidget

cache_location = "data"
input_path = f"{cache_location}/input"

# Evo app credentials
client_id = "<your-client-id>"  # Replace with your client ID
redirect_url = "<your-redirect-url>"  # Replace with your redirect URL

client_id = "daves-evo-client"
redirect_url = "http://localhost:32369/auth/callback"

manager = await ServiceManagerWidget.with_auth_code(
    discovery_url="https://discover.api.seequent.com",
    redirect_url=redirect_url,
    client_id=client_id,
    cache_location=cache_location,
).login()

c:\Users\david.knight\Downloads\forks\evo-data-converters\samples\duf-jupyter-conversion\.venv\Lib\site-packages\evo\notebooks\authorizer.py:108: UserWarning: The evo.notebooks.AuthorizationCodeAuthorizer is not secure, and should only ever be used in Jupyter notebooks in a private environment.
  warnings.warn(


ServiceManagerWidget(children=(VBox(children=(HBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR…

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import tkinter as tk
from tkinter import filedialog
import os
from pathlib import Path

# Create widgets
select_button = widgets.Button(
    description='Select DUF File',
    button_style='primary',
    tooltip='Click to select a .duf file'
)

output_label = widgets.Label(value='No file selected')
error_label = widgets.Label(value='', style={'text_color': 'red'})

# Variable to store selected file path
selected_file_path = None

# Preload saved selection from .env (if present)
env_file_path = Path(cache_location) / '.env'
os.makedirs(cache_location, exist_ok=True)

saved_path = None
if env_file_path.exists():
    try:
        with open(env_file_path, 'r') as f:
            for line in f:
                if line.startswith('SELECTED_DUF_FILE='):
                    saved_path = line.strip().split('=', 1)[1].strip().strip('"')
    except Exception as e:
        error_label.value = f'ERROR: Unable to read .env: {e}'

if saved_path:
    p = Path(saved_path)
    if p.suffix.lower() != '.duf':
        error_label.value = 'ERROR: Saved file is not a .duf file'
    elif p.exists():
        selected_file_path = str(p)
        output_label.value = f'Selected: {p.name}'
    else:
        error_label.value = 'ERROR: Saved file not found on disk'


def on_button_click(b):
    """Handle button click to open file dialog"""
    global selected_file_path
    
    error_label.value = ''
    output_label.value = 'Opening file dialog...'
    
    # Create file dialog
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    root.attributes('-topmost', True)  # Bring dialog to front
    
    file_path = filedialog.askopenfilename(
        title="Select DUF File",
        filetypes=[("DUF Files", "*.duf"), ("All Files", "*.*")]
    )
    
    root.destroy()
    
    if not file_path:
        output_label.value = 'No file selected'
        return
    
    file_path = Path(file_path)
    
    # Validate file extension
    if file_path.suffix.lower() != '.duf':
        error_label.value = 'ERROR: Invalid file type. Only .duf files are allowed.'
        output_label.value = 'No file selected'
        return
    
    # Store the selected file path
    selected_file_path = str(file_path)
    
    # Ensure only one SELECTED_DUF_FILE entry in .env
    existing_lines = []
    if env_file_path.exists():
        with open(env_file_path, 'r') as f:
            for line in f:
                if not line.startswith('SELECTED_DUF_FILE='):
                    existing_lines.append(line)
    existing_lines.append(f'SELECTED_DUF_FILE={selected_file_path}\n')
    with open(env_file_path, 'w') as f:
        f.writelines(existing_lines)
    
    # Display success
    output_label.value = f'Selected: {file_path.name}'
    print(f'Full path: {selected_file_path}')
    print(f'Updated {env_file_path} with unique SELECTED_DUF_FILE entry')

# Attach event handler
select_button.on_click(on_button_click)

# Display the widget
display(widgets.VBox([
    select_button,
    output_label,
    error_label
]))